In [14]:
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
import re
from transformers import AutoTokenizer


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Dataset

In [2]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
ds = load_dataset("cardiffnlp/databench", "qa")
print(ds)  # This will show you all the available splits in the dataset


Resolving data files:   0%|          | 0/65 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/65 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'type', 'columns_used', 'column_types', 'sample_answer', 'dataset'],
        num_rows: 1308
    })
})


In [7]:
# Convert the entire dataset into a DataFrame
df = pd.DataFrame(ds['train'])


Data configuration

In [8]:
df

,question,answer,type,columns_used,column_types,sample_answer,dataset
0,Is the person with the highest net worth self-...,True,boolean,"['finalWorth', 'selfMade']","['number[uint32]', 'boolean']",False,001_Forbes
1,Does the youngest billionaire identify as male?,True,boolean,"['age', 'gender']","['number[UInt8]', 'category']",True,001_Forbes
2,Is the city with the most billionaires in the ...,True,boolean,"['city', 'country']","['category', 'category']",True,001_Forbes
3,Is there a non-self-made billionaire in the to...,True,boolean,"['rank', 'selfMade']","['number[uint16]', 'boolean']",False,001_Forbes
4,Does the oldest billionaire have a philanthrop...,False,boolean,"['age', 'philanthropyScore']","['number[UInt8]', 'number[UInt8]']",False,001_Forbes
...,...,...,...,...,...,...,...
1303,Which 2 countries made the least transactions?,"['Saudi Arabia', 'Bahrain']",list[category],['Country'],['category'],"['United Kingdom', 'Australia']",065_RFM
1304,What are the invoice numbers of the top 5 tran...,"['581483', '541431', '578841', '542504', '5730...",list[number],"['InvoiceNo', 'Quantity']","['number[UInt32]', 'number[int32]']","['548005', '555200', '573399', '563745', '5689...",065_RFM
1305,What are the stock codes of the bottom 5 trans...,"[23843, 23166, 23005, 23005, 84347]",list[number],"['InvoiceNo', 'Quantity']","['number[UInt32]', 'number[int32]']","['22465', '21109', '82484', '22799', '82600']",065_RFM
1306,What are the four most commonly ordered quanti...,"[1, 2, 12, 6]",list[number],['Quantity'],['number[int32]'],"[1, 4, 2, 12]",065_RFM


In [13]:
# View the first few rows of the DataFrame
print("\n\nhead\n", df.head())

# Check basic information about the DataFrame
print("\n\nInformation\n", df.info())

# Display summary statistics of numeric columns (if any)
print("\n\nDescribe\n", df.describe())

# View column names and their types
print("\n\nTypes\n", df.dtypes)

# Check the distribution of the 'type' column
print(df['type'].value_counts())

# Check for missing values
print("\n\nIs null\n", df.isnull().sum())




head
                                             question answer     type  \
0  Is the person with the highest net worth self-...   True  boolean   
1    Does the youngest billionaire identify as male?   True  boolean   
2  Is the city with the most billionaires in the ...   True  boolean   
3  Is there a non-self-made billionaire in the to...   True  boolean   
4  Does the oldest billionaire have a philanthrop...  False  boolean   

                   columns_used                        column_types  \
0    ['finalWorth', 'selfMade']       ['number[uint32]', 'boolean']   
1             ['age', 'gender']       ['number[UInt8]', 'category']   
2           ['city', 'country']            ['category', 'category']   
3          ['rank', 'selfMade']       ['number[uint16]', 'boolean']   
4  ['age', 'philanthropyScore']  ['number[UInt8]', 'number[UInt8]']   

  sample_answer     dataset  
0         False  001_Forbes  
1          True  001_Forbes  
2          True  001_Forbes  
3         Fa

Data preprocessing

- Missing values are handled for both answer and columns_used columns.
- Text preprocessing (e.g., converting to lowercase and removing punctuation).
- Tokenization using Hugging Face’s tokenizer.
- Categorical columns (type, dataset) are converted to category type.
- One-hot encoding applied to categorical columns.
- Answer encoding (if needed) using LabelEncoder.Missing values are handled for both answer and columns_used columns.
- Text preprocessing (e.g., converting to lowercase and removing punctuation).
- Tokenization using Hugging Face’s tokenizer.
- Categorical columns (type, dataset) are converted to category type.
- One-hot encoding applied to categorical columns.
- Answer encoding using LabelEncoder.

In [27]:
# 1. Handle missing values
df.dropna(subset=['answer'], inplace=True)
df['columns_used'] = df['columns_used'].fillna("Unknown")
df['sample_answer'] = df['sample_answer'].fillna("Unknown")

# 2. Text Preprocessing
df['question'] = df['question'].apply(lambda x: re.sub(r'[^\w\s]', '', x.lower()))  # Lowercase and remove punctuation

# 3. Tokenization using Hugging Face's tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
df['question_tokenized'] = df['question'].apply(lambda x: tokenizer(x, padding=True, truncation=True, max_length=128))
df['answer_tokenized'] = df['answer'].apply(lambda x: tokenizer(x, padding=True, truncation=True, max_length=128))

# 4. Process 'columns_used' column to separate multiple columns and convert lists to strings
df['columns_used'] = df['columns_used'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# 5. Handle categorical columns (e.g., 'type', 'dataset')
df['type'] = df['type'].astype('category')
df['dataset'] = df['dataset'].astype('category')

# 6. Encoding answers (if needed)
le = LabelEncoder()
df['answer_encoded'] = le.fit_transform(df['answer'])

# 7. Apply one-hot encoding to categorical columns
X = pd.get_dummies(df[['question', 'type', 'columns_used']], columns=['type', 'columns_used'], drop_first=True)

# 8. Show the first few rows of the processed data
print(X.head())


                                            question  type_category  \
0  is the person with the highest net worth selfmade          False   
1     does the youngest billionaire identify as male          False   
2  is the city with the most billionaires in the ...          False   
3  is there a nonselfmade billionaire in the top ...          False   
4  does the oldest billionaire have a philanthrop...          False   

   type_list[category]  type_list[number]  type_number  \
0                False              False        False   
1                False              False        False   
2                False              False        False   
3                False              False        False   
4                False              False        False   

   columns_used_ [Incident_Location  \
0                             False   
1                             False   
2                             False   
3                             False   
4                            